In [2]:
import os
import pandas as pd
import plotly.graph_objects as go

### Create a fig folder to store the analysis figures

In [3]:
if not os.path.exists("fig"):
    os.makedirs("fig")

## Graph Requirements

- Queries per second (QPS) on the x-axis (the x-axis should range from 0 to 80K). (note: the actual achieved QPS, not the target QPS)
- 95th percentile latency on the y-axis (the y-axis should range from 0 to 6 ms).
- Label your axes.
- 7 lines, one for each configuration. Add a legend.
- State how many runs you averaged across and include error bars at each point in both
dimensions

### There are 7 configurations
- without interference &rarr; some-memcached_run.txt
- cpu &rarr; ibench-cpu_run.txt
- l1d &rarr; ibench-l1d_run.txt
- l1i &rarr; ibench-l1i_run.txt
- l2 &rarr; ibench-l2_run.txt
- llc &rarr; ibench-llc_run.txt
- membw &rarr; ibench-membw_run-1.txt

In [4]:
categories = ["without_intereference", "cpu", "l1d", "l1i", "l2", "llc", "membw"]
benchmark_file = {}

for category in categories:
    benchmark_file[category] = []
    if category == "without_intereference":
        benchmark_file[category].append(f"some-memcached")
    else:
        benchmark_file[category].append(f"ibench-{category}")


In [5]:
dir_path = "benchmarks"
file_list = os.listdir(dir_path)

df_list = []
for filename in file_list:
    df = pd.read_csv(f"{dir_path}/{filename}", nrows=16, header=1, sep="\\s+")
    df["filename"] = filename.split("_")[0]
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df

,#type,avg,std,min,p5,p10,p50,p67,p75,p80,p85,p90,p95,p99,p999,p9999,QPS,target,filename
0,read,219.6,85.4,86.8,148.9,160.8,211.9,233.0,245.8,254.5,267.6,283.4,307.6,368.0,949.6,2883.7,4983.6,5000,some-memcached
1,read,248.4,98.1,86.8,151.1,170.1,240.4,267.9,283.9,295.2,309.8,327.6,357.3,433.6,1519.7,2374.4,10044.8,10000,some-memcached
2,read,247.5,99.7,74.4,133.9,155.3,243.3,273.8,291.1,303.5,317.5,337.0,367.2,438.2,615.9,3272.2,15070.0,15000,some-memcached
3,read,262.1,98.3,74.4,129.3,149.3,256.5,293.4,313.7,328.6,345.9,370.3,412.3,514.1,928.3,1911.1,19997.8,20000,some-memcached
4,read,281.8,117.0,74.4,129.3,149.1,275.0,317.5,340.9,358.3,378.3,407.2,456.5,595.4,1520.6,1993.5,24889.6,25000,some-memcached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,read,791.7,494.9,68.9,191.8,263.0,819.2,940.7,993.2,1029.9,1080.9,1188.4,1586.7,2012.7,7279.2,8470.6,59892.4,60000,ibench-l1d
332,read,1119.8,524.2,86.8,791.7,847.3,1026.6,1084.8,1124.1,1158.6,1205.2,1454.2,1946.0,2274.8,7897.7,9657.4,59484.5,65000,ibench-l1d
333,read,1122.8,517.6,80.4,796.3,849.5,1026.5,1086.5,1130.1,1166.7,1230.7,1466.6,1939.4,2290.4,7873.1,9616.0,59550.8,70000,ibench-l1d
334,read,1113.8,512.7,148.8,810.0,854.8,1019.1,1075.0,1102.5,1142.2,1182.0,1390.8,1933.7,2221.8,7862.7,11907.6,60116.2,75000,ibench-l1d


In [6]:
agg_df = df.groupby(['filename', 'target']).agg(
    p95_mean=('p95', 'mean'),
    p95_std=('p95', 'std'),
    QPS_mean=('QPS', 'mean'),
    QPS_std=('QPS', 'std'),
).reset_index()

print(agg_df.head())

     filename  target     p95_mean     p95_std      QPS_mean     QPS_std
0  ibench-cpu    5000   921.366667  283.519123   4992.733333   16.531586
1  ibench-cpu   10000  1178.366667  153.747271  10004.066667   88.635960
2  ibench-cpu   15000  1318.766667   29.139378  14968.333333   91.662497
3  ibench-cpu   20000  1341.566667    8.727161  19997.200000   60.004000
4  ibench-cpu   25000  1319.933333   72.992899  25065.166667  165.477018


In [18]:
fig = go.Figure()

for fname in agg_df["filename"].unique():
    sub_df = agg_df[agg_df["filename"] == fname]

    fig.add_trace(go.Scatter(
        x = sub_df["QPS_mean"],
        y = sub_df["p95_mean"]/1000,
        mode = "lines+markers",
        name = fname,
        marker = dict(size = 8),
        line=dict(width=2),
        error_x=dict(
            type="data",
            array=sub_df["QPS_std"],
            thickness=1.5,
            color='gray',
            width=4
        ),
        error_y=dict(
            type="data",
            array=sub_df["p95_std"]/1000,
            thickness=1.5,
            color='gray',
            width=4
        ),
        text=sub_df['target'],
        hovertemplate=(
        "Achieved QPS: %{x}<br>" +
        "95th Latency: %{y} ms<br>" +
        "Target QPS: %{text}<extra></extra>"
        )
    ))

# Layout setting
fig.update_layout(
    title = "Part 1: 95 Percentile Latency vs Achieved QPS",
    xaxis_title = "QPS (actual achieved rather than target)",
    yaxis_title="95 Percentile Latency (ms)",
    xaxis = dict(range=[0, 80000]),
    yaxis = dict(range=[0, 6]),
    legend=dict(
        x=0.99,
        y=0.99,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12)
    )
)

fig.add_annotation(
    text="Averaged across 3 runs",
    xref="paper", yref="paper",
    x=0.99, y=0.01,
    showarrow=False,
    font=dict(size=12),
    align="right"
)

fig.show()

Save the figure

In [19]:
# It seems that I can't store the image locally but the html works
# fig.write_image("fig/95_percentile_latency_achieved_qps.png")
fig.write_html("fig/95_percentile_latency_achieved_qps.html")